In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import re
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_train = pd.read_csv('train.csv', sep='\t', encoding='utf8', index_col='id')

In [3]:
morph=pymorphy2.MorphAnalyzer()

In [4]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words

In [5]:
%%time
title_tokens=[]
text_tokens=[]
pos_list = []
for i in df_train.index:
    if i % 10000 == 0:
        print(i,"\n")
    if df_train.iloc[i:i+1,2][i] == 1:
        bs=BeautifulSoup(df_train.iloc[i:i+1,1][i], "html5lib")
        title_tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', df_train.iloc[i:i+1,0][i])
        text_tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', BeautifulSoup(" ".join([p.text for p in bs.find_all("li")]), "html5lib").get_text())
        tokens=title_tokens+text_tokens
        for t in tokens:
            pv=morph.parse(t)
            for p in pv:
                if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    pos_list.append(p.normal_form)
                    break

0 

10000 

20000 

30000 

40000 

50000 

60000 

70000 

80000 

90000 

100000 

110000 

120000 

130000 

140000 

150000 

160000 

170000 

180000 

190000 

Wall time: 33min 47s


In [6]:
pos_cc=[' '.join(pos_list)]

In [7]:
pos_lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

In [8]:
pos_matrix=pos_lemmaCounter.fit_transform(pos_cc)

In [9]:
pos_matrix.shape

(1, 536419)

In [10]:
%%time
title_tokens=[]
text_tokens=[]
neg_list = []
for i in df_train.index:
    if i % 10000 == 0:
        print(i,"\n")
    if df_train.iloc[i:i+1,2][i] == 0:
        bs=BeautifulSoup(df_train.iloc[i:i+1,1][i], "html5lib")
        title_tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', df_train.iloc[i:i+1,0][i])
        text_tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', BeautifulSoup(" ".join([p.text for p in bs.find_all("li")]), "html5lib").get_text())
        tokens=title_tokens+text_tokens
        for t in tokens:
            pv=morph.parse(t)
            for p in pv:
                if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    neg_list.append(p.normal_form)
                    break

0 

10000 

20000 

30000 

40000 

50000 

60000 

70000 

80000 

90000 

100000 

110000 

120000 

130000 



C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


140000 

150000 

160000 

170000 

180000 

190000 

Wall time: 31min 48s


In [11]:
neg_cc=[' '.join(neg_list)]

In [12]:
neg_lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

In [13]:
neg_matrix=neg_lemmaCounter.fit_transform(neg_cc)

In [14]:
neg_matrix.shape

(1, 816355)

In [15]:
pos_zero_matrix = np.zeros((pos_matrix.shape[0],pos_matrix.shape[1]))
neg_zero_matrix = np.zeros((neg_matrix.shape[0],neg_matrix.shape[1]))

In [16]:
pos_zero_matrix.shape

(1, 536419)

In [17]:
neg_zero_matrix.shape

(1, 816355)

In [18]:
df_test = pd.read_csv('test.csv', sep='\t', encoding='utf8', index_col='id')

In [19]:
df_test.shape

(170179, 2)

In [20]:
test_pos_zero_matrix = np.zeros((pos_matrix.shape[0],pos_matrix.shape[1]))

In [21]:
test_pos_zero_matrix.shape

(1, 536419)

In [22]:
df_test.iloc[:1,1][200000]

'<p><strong>Обязанности:</strong></p> <ul> <li>Работа с клиентом в салоне,выезд на замер ,создание дизайн-проекта,расчеты,ведение документации,заключение договоров</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>Опыт работы желателен,уверенный пользователь ПК,грамотная речь,желание учиться и развиваться</li> </ul> <p> </p> <p><strong>Условия:</strong></p> <ul> <li>Трудоустройство по ТК, обучение+стажировка,оклад+% от личных продаж,удобный график</li> </ul>'

In [23]:
test_lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
bs_test=BeautifulSoup(df_test.iloc[:1,1][200000], "html5lib")
test_row=[' '.join(getMeaningfullWords(df_test.iloc[:1,0][200000]+' '+BeautifulSoup(" ".join([p.text for p in bs_test.find_all("li")]), "html5lib").get_text()))]
test_row_matrix=test_lemmaCounter.fit_transform(test_row)

In [24]:
test_row_matrix.shape

(1, 64)

In [25]:
for w in test_lemmaCounter.vocabulary_:
    if str(pos_lemmaCounter.vocabulary_.get(w)) != 'None':
        test_pos_zero_matrix[0,pos_lemmaCounter.vocabulary_[w]] = test_row_matrix[0,test_lemmaCounter.vocabulary_[w]]

In [26]:
test_lemmaCounter.vocabulary_[w]

63

In [27]:
pos_lemmaCounter.vocabulary_[w]

488832

In [28]:
test_pos_zero_sparse_matrix = sparse.csr_matrix(test_pos_zero_matrix)

In [29]:
res_array = cosine_similarity(pos_matrix, test_pos_zero_sparse_matrix)
print(res_array)

[[0.39948663]]


In [30]:
res_array[0][0]

0.3994866347084164

In [31]:
test_neg_zero_matrix = np.zeros((neg_matrix.shape[0],neg_matrix.shape[1]))

In [32]:
test_neg_zero_matrix.shape

(1, 816355)

In [33]:
for w in test_lemmaCounter.vocabulary_:
    if str(neg_lemmaCounter.vocabulary_.get(w)) != 'None':
        test_neg_zero_matrix[0,neg_lemmaCounter.vocabulary_[w]] = test_row_matrix[0,test_lemmaCounter.vocabulary_[w]]

In [34]:
test_neg_zero_sparse_matrix = sparse.csr_matrix(test_neg_zero_matrix)

In [35]:
neg_res_array = cosine_similarity(neg_matrix, test_neg_zero_sparse_matrix)
print(neg_res_array)

[[0.34450664]]


In [36]:
neg_res_array[0][0]

0.344506636514514

In [37]:
df_test.index[:5][1]

200001

In [39]:
%%time
pos_zero_matrix = np.zeros((pos_matrix.shape[0],pos_matrix.shape[1]))
neg_zero_matrix = np.zeros((neg_matrix.shape[0],neg_matrix.shape[1]))
res_lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
j = 0
result_df = pd.DataFrame(columns=['target'], index=df_test.index)
for i in df_test.index:
    if j % 10000 == 0:
        print(j,"\n")
    bs_test=BeautifulSoup(df_test.iloc[j:j+1,1][i], "html5lib")
    test_row=[' '.join(getMeaningfullWords(df_test.iloc[j:j+1,0][i]+' '+BeautifulSoup(" ".join([p.text for p in bs_test.find_all("li")]), "html5lib").get_text()))]
    if len(test_row[0]) > 0:
        test_row_matrix=res_lemmaCounter.fit_transform(test_row)
        pos_zero_matrix.fill(0)
        for w in res_lemmaCounter.vocabulary_:
            if str(pos_lemmaCounter.vocabulary_.get(w)) != 'None':
                pos_zero_matrix[0,pos_lemmaCounter.vocabulary_[w]] = test_row_matrix[0,res_lemmaCounter.vocabulary_[w]]
        pos_zero_sparse_matrix = sparse.csr_matrix(pos_zero_matrix)
        pos_res_array = cosine_similarity(pos_matrix, pos_zero_sparse_matrix)
        neg_zero_matrix.fill(0)
        for w in res_lemmaCounter.vocabulary_:
            if str(neg_lemmaCounter.vocabulary_.get(w)) != 'None':
                neg_zero_matrix[0,neg_lemmaCounter.vocabulary_[w]] = test_row_matrix[0,res_lemmaCounter.vocabulary_[w]]
        neg_zero_sparse_matrix = sparse.csr_matrix(neg_zero_matrix)
        neg_res_array = cosine_similarity(neg_matrix, neg_zero_sparse_matrix)
        if pos_res_array[0][0] > neg_res_array[0][0]:
            result_df.loc[i:i+1, 'target'] = 1
        else:
            result_df.loc[i:i+1, 'target'] = 0
    else:
        result_df.loc[i:i+1, 'target'] = 0
    j += 1

0 

10000 

20000 

30000 

40000 

50000 

60000 

70000 

80000 

90000 

100000 

110000 

120000 

130000 

140000 

150000 

160000 

170000 

Wall time: 3h 5min 48s


In [40]:
result_df[result_df['target']==1].count()

target    85686
dtype: int64

In [41]:
result_df[result_df['target']==0].count()

target    84493
dtype: int64

In [42]:
result_df.head(100)

,target
id,
200000,1
200001,1
200002,1
200003,1
200004,0
200005,0
200006,1
200007,1
200008,0


In [44]:
result_df.to_csv('submission.csv', sep=',', encoding='utf-8', index='id')